<a href="https://colab.research.google.com/github/davidrimon2004/DEPI_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem Formulation

##Problem Description:
This project aims to develop a machine learning model that predicts future sales
and demand by utilising historical sales data and external factors, including
product details, promotions, seasonality, holidays, and economic indicators. The
goal is to analyse historical patterns and generate reliable forecasts that help
businesses make data-driven decisions to reduce costs, increase efficiency, and
improve customer satisfaction by predicting the daily sales for the next 28 days.

##Objectives

● Collect and preprocess historical sales and demand data.

● Identify key features that influence sales trends.

● Build, train, and optimise forecasting models to predict future sales and
demand.

● Deploy the best-performing model to generate forecasts in real-time or in
batches.
##Data source:
Hierarchical sales data from Walmart, the world’s largest company by revenue in the US.



# **Setup for google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Important libraries**

In [2]:
import pandas as pd

In [4]:
sales_validation=pd.read_csv('/content/drive/MyDrive/DEPI_project/data/sales_train_validation.csv')
sales_validation.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1


# Schema formating

In [5]:
data = sales_validation.melt(
    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep
    var_name="d",                          # new column name for day labels (d_1, d_2, ...)
    value_name="sales"                     # new column name for sales values
)



In [6]:
data.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [7]:
data.drop(columns=['id'],inplace=True)

In [10]:
data.head(200)

,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...
195,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2
196,HOBBIES_1_203,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
197,HOBBIES_1_204,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
198,HOBBIES_1_205,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [12]:
cal=pd.read_csv('/content/drive/MyDrive/DEPI_project/data/calendar.csv')
cal.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0


In [13]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [14]:
cal["date"]= pd.to_datetime(cal["date"])

In [15]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


In [16]:
cal.head(200)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2011-08-12,11128,Friday,7,8,2011,d_196,NaN,NaN,NaN,NaN,0,1,1
196,2011-08-13,11129,Saturday,1,8,2011,d_197,NaN,NaN,NaN,NaN,0,1,0
197,2011-08-14,11129,Sunday,2,8,2011,d_198,NaN,NaN,NaN,NaN,0,0,1
198,2011-08-15,11129,Monday,3,8,2011,d_199,NaN,NaN,NaN,NaN,0,1,1


In [17]:
cal["event_name_1"]= cal["event_name_1"].fillna("No event")
cal["event_type_1"]= cal["event_type_1"].fillna("No event")
cal["event_name_2"]= cal["event_name_2"].fillna("No event")
cal["event_type_2"]= cal["event_type_2"].fillna("No event")

In [18]:
cal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  1969 non-null   object        
 8   event_type_1  1969 non-null   object        
 9   event_name_2  1969 non-null   object        
 10  event_type_2  1969 non-null   object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


In [20]:
import numpy as np

# Merge data and cal dataframes on the 'd' column
merged_data = pd.merge(data, cal, on='d', how='left')

conditions = [
    merged_data["state_id"] == "CA",
    merged_data["state_id"] == "TX",
    merged_data["state_id"] == "WI"
]
choices= [
    merged_data["snap_CA"],
    merged_data["snap_TX"],
    merged_data["snap_WI"]
]
merged_data["snap"]= np.select(conditions, choices)

# Now you can use merged_data instead of data and cal separately for further analysis
display(merged_data.head())

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0


In [27]:
merged_data.head(199999)

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,snap
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,...,1,2011,No event,No event,No event,No event,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,FOODS_1_201,FOODS_1,FOODS,TX_2,TX,d_7,1,2011-02-04,11101,Friday,...,2,2011,No event,No event,No event,No event,1,0,0,0
199995,FOODS_1_202,FOODS_1,FOODS,TX_2,TX,d_7,0,2011-02-04,11101,Friday,...,2,2011,No event,No event,No event,No event,1,0,0,0
199996,FOODS_1_203,FOODS_1,FOODS,TX_2,TX,d_7,3,2011-02-04,11101,Friday,...,2,2011,No event,No event,No event,No event,1,0,0,0
199997,FOODS_1_204,FOODS_1,FOODS,TX_2,TX,d_7,0,2011-02-04,11101,Friday,...,2,2011,No event,No event,No event,No event,1,0,0,0
